In [ ]:
# Path setup / Imports
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
    
import yaml
import os
from transition import RNNTransition
from utils import save_yaml

In [ ]:
# Start training

yaml_path = "config/training/train_rnn_helm.yaml"
with open(os.path.join(repo_root, yaml_path)) as f:
    conf = yaml.safe_load(f)
output_dir = os.path.join(repo_root, conf.pop("output_dir"))

model, best_state_dict, lang = RNNTransition.train_rnn_from_conf(conf)

In [ ]:
# Save models, setting and lang

model.save(os.path.join(output_dir, "last"))
model.load_state_dict(best_state_dict)
model.save(os.path.join(output_dir, "best"))

def save_conf_and_lang(output_dir, conf, lang):
    for root, dirs, files in os.walk(output_dir):
        if not dirs: # bottom
            save_yaml(conf=conf, output_dir=root)
            lang.save(os.path.join(root, "language.lang"))

save_conf_and_lang(output_dir, conf, lang)